In [5]:
import ibm_db

In [2]:
#Replace the placeholder values with your actual Db2 hostname, username, and password:
dsn_hostname = "55fbc997-9266-4331-afd3-888b05e734c0.bs2io90l08kqb1od8lcg.databases.appdomain.cloud" # e.g.: "54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud"
dsn_uid = "nzx60122"        # e.g. "abc12345"
dsn_pwd = "JbJHiCcZ6EkJAMMe"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "31929"                # e.g. "32733" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_security = "SSL"              #i.e. "SSL"

In [3]:
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

#print the connection string to check correct values are specified
print(dsn)

DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=55fbc997-9266-4331-afd3-888b05e734c0.bs2io90l08kqb1od8lcg.databases.appdomain.cloud;PORT=31929;PROTOCOL=TCPIP;UID=nzx60122;PWD=JbJHiCcZ6EkJAMMe;SECURITY=SSL;


In [6]:

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database:  BLUDB as user:  nzx60122 on host:  55fbc997-9266-4331-afd3-888b05e734c0.bs2io90l08kqb1od8lcg.databases.appdomain.cloud


In [34]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [7]:
selectQuery = "select * from Census_Data"

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

#Fetch the Dictionary (for the first row only)
ibm_db.fetch_both(selectStmt)

{'COMMUNITY_AREA_NUMBER': 1,
 0: 1,
 'COMMUNITY_AREA_NAME': 'Rogers Park',
 1: 'Rogers Park',
 'PERCENT_OF_HOUSING_CROWDED': '7.7',
 2: '7.7',
 'PERCENT_HOUSEHOLDS_BELOW_POVERTY': '23.6',
 3: '23.6',
 'PERCENT_AGED_16__UNEMPLOYED': '8.7',
 4: '8.7',
 'PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA': '18.2',
 5: '18.2',
 'PERCENT_AGED_UNDER_18_OR_OVER_64': '27.5',
 6: '27.5',
 'PER_CAPITA_INCOME': 23939,
 7: 23939,
 'HARDSHIP_INDEX': 39,
 8: 39}

In [10]:
import pandas
import ibm_db_dbi

#connection for pandas
pconn = ibm_db_dbi.Connection(conn)

selectQuery = "select * from Census_Data"
pdf = pandas.read_sql(selectQuery, pconn)
pdf

,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_OF_HOUSING_CROWDED,PERCENT_HOUSEHOLDS_BELOW_POVERTY,PERCENT_AGED_16__UNEMPLOYED,PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA,PERCENT_AGED_UNDER_18_OR_OVER_64,PER_CAPITA_INCOME,HARDSHIP_INDEX
0,1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
2,3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0
3,4.0,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17.0
4,5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0
...,...,...,...,...,...,...,...,...,...
73,74.0,Mount Greenwood,1.0,3.4,8.7,4.3,36.8,34381,16.0
74,75.0,Morgan Park,0.8,13.2,15.0,10.8,40.3,27149,30.0
75,76.0,O'Hare,3.6,15.4,7.1,10.9,30.3,25828,24.0
76,77.0,Edgewater,4.1,18.2,9.2,9.7,23.8,33385,19.0


__Problem 1: Find the total number of crimes recorded in the CRIME table.__

In [11]:
selectQuery1 = "select count(*) from CHICAGO_CRIME_DATA"
pdf1 = pandas.read_sql(selectQuery1, pconn)
pdf1

,1
0,499


__Problem 2: List community areas with per capita income less than 11000.__

In [12]:
selectQuery2 = "select community_area_name from Census_Data where per_capita_income<11000"
pdf2 = pandas.read_sql(selectQuery2, pconn)
pdf2

,COMMUNITY_AREA_NAME
0,West Garfield Park
1,South Lawndale
2,Fuller Park
3,Riverdale


__Problem 3: List all case numbers for crimes involving minors?__

In [13]:
selectQuery3 = "select case_number from CHICAGO_CRIME_DATA where description like '%MINOR%'"
pdf3 = pandas.read_sql(selectQuery3, pconn)
pdf3

,CASE_NUMBER
0,HY412341


__Problem 4: List all kidnapping crimes involving a child?(children are not considered minors for the purposes of crime analysis)__

In [51]:
selectQuery4 = "select * from CHICAGO_CRIME_DATA where primary_type like '%KIDNAP%' and description like '%CHILD%'"
pdf4 = pandas.read_sql(selectQuery4, pconn)
pdf4

,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,...,WARD,COMMUNITY_AREA,FBI_CODE,X_COORDINATE,Y_COORDINATE,YEAR,UPDATED_ON,LATITUDE,LONGITUDE,LOCATION


Since i have used smaller sample containing only 500 rows, the data might not have the result for above query.

__Problem 5: What kind of crimes were recorded at schools?__

In [14]:
selectQuery5 = "select distinct(primary_type) from CHICAGO_CRIME_DATA where location_description like 'SCHOOL%'"
pdf5 = pandas.read_sql(selectQuery5, pconn)
pdf5

,PRIMARY_TYPE


THE DATASET WHICH I HAVE USED DOES NOT HAVE SCHOOLS IN LOCATION_DESCRIPTION COLUMN

__Problem 6: List the average safety score for all types of schools.__

In [17]:
selectQuery6 = "select 'Elementary,Middle or High School', AVG(safety_score) from CPS_Report_Cards GROUP BY 'Elementary,Middle or High School'"
pdf6 = pandas.read_sql(selectQuery6, pconn)
pdf6

,1,2
0,"Elementary,Middle or High School",49


__Problem 7: List 5 community areas with highest % of households below poverty line.__

In [19]:
selectQuery7 = "select community_area_name, percent_households_below_poverty from Census_Data order by percent_households_below_poverty desc limit 5"
pdf7 = pandas.read_sql(selectQuery7, pconn)
pdf7 

,COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
0,Riverdale,56.5
1,Fuller Park,51.2
2,Englewood,46.6
3,North Lawndale,43.1
4,East Garfield Park,42.4


__Problem 8: Which community area(number) is most crime prone?__

In [22]:
selectQuery8 = "select Community_Area, count(*) as frequency from CHICAGO_CRIME_DATA group by Community_Area order by frequency desc limit 1"
pdf8 = pandas.read_sql(selectQuery8, pconn)
pdf8 

,COMMUNITY_AREA,FREQUENCY
0,25,29


__Problem 9: Use a sub-query to find the name of the community area with highest hardship index.__

In [29]:
selectQuery9 = "Select community_area_name from Census_Data where hardship_index= (Select max(hardship_index) from Census_Data)"
pdf9 = pandas.read_sql(selectQuery9, pconn)
pdf9

,COMMUNITY_AREA_NAME
0,Riverdale


__Problem 10: Use a sub-query to determine the Community Area Name with most number of crimes?__

In [33]:
Query10="Select community_area_name from Census_Data where community_area_number=(Select community_area from CHICAGO_CRIME_DATA where community_area = 25 limit 1)"
pdf10 = pandas.read_sql(Query10, pconn)
pdf10

,COMMUNITY_AREA_NAME
0,Austin
